In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Handle class imbalance

In [4]:

from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

### Dagshub setup

In [8]:
import dagshub
dagshub.init(repo_owner='akibnayan182', repo_name='mlflow-dagshub-demo', mlflow=True)


Accessing as akibnayan182

Initialized MLflow to track repo "akibnayan182/mlflow-dagshub-demo"

Repository akibnayan182/mlflow-dagshub-demo initialized!

In [9]:
# Initialize MLflow
# mlflow.set_experiment("Anomaly Detection 1")
# For Local mlflow server
# mlflow.set_tracking_uri("http://localhost:5000")

# For cloud mlflow dagshub server
# mlflow.set_tracking_uri("https://dagshub.com/akibnayan182/mlflow-dagshub-demo.mlflow")


import os

os.environ['MLFLOW_TRACKING_USERNAME'] = 'akibnayan182'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '5e6432a919043ad2ea3e173bb047e25ca33a2827'
mlflow.set_tracking_uri("https://dagshub.com/akibnayan182/mlflow-dagshub-demo.mlflow")


#print("Tracking URI:", mlflow.get_tracking_uri()) 
mlflow.set_experiment("Anomaly Detection 9")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "xgb-models")
        else:
            mlflow.sklearn.log_model(model, "sklearn-models")  

### Register the Model

In [10]:
model_name = 'xgboost'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/{model_name}'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name="xgb-model")

Registered model 'xgb-model' already exists. Creating a new version of this model...
2025/10/08 20:55:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb-model, version 2
Created version '2' of model 'xgb-model'.


### Load the Model

In [14]:
os.environ["MLFLOW_TRACKING_USERNAME"] = "akibnayan182"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "5e6432a919043ad2ea3e173bb047e25ca33a2827"
mlflow.set_tracking_uri("https://dagshub.com/akibnayan182/mlflow-dagshub-demo.mlflow")

run_id = "9dfc190d51744cabb8ec6ce5366868e0"
model_path = "xgb-models"
model_uri = f"runs:/{run_id}/{model_path}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [21]:
model_path = "xgb-model"
production_model_name = "anomaly-detection-production"

client = mlflow.MlflowClient()

client.copy_model_version(
    src_model_uri=f"models:/{model_path}@challenger",
    dst_name=production_model_name
)

<ModelVersion: aliases=[], creation_timestamp=1759938823683, current_stage='None', description='', last_updated_timestamp=1759938823683, name='anomaly-detection-production', run_id='9dfc190d51744cabb8ec6ce5366868e0', run_link='', source='models:/xgb-model/2', status='READY', status_message='', tags={}, user_id='', version='1'>